In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import keras
import numpy as np
import cv2 as cv
import sys

print(sys.version)
print(cv.__version__)
print(np.__version__)
print(tf.__version__)
print(keras.__version__)

In [ ]:
import zipfile

# 파일 경로 입력
zip_file_name = '' # 압축을 풀 zip파일의 경로를 입력하세요

# 압축 해제할 경로 입력
extraction_dir = '/content/dataset' # 압축해제할 경로를 입력하세요

# 압축 해제
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)

In [ ]:
from imutils import paths

search_dir = "/content/dataset/emotion_images/data" #train data를 불러올 경로를 입력하세요

search_dir_anger = "/content/dataset/emotion_images/data/anger"

search_dir_happy = "/content/dataset/emotion_images/data/happy"

search_dir_normal = "/content/dataset/emotion_images/data/normal"
search_dir_sad = "/content/dataset/emotion_images/data/sad"
search_dir_worry = "/content/dataset/emotion_images/data/worry"

image_paths = sorted(
    list(paths.list_images(search_dir))
)

image_paths_anger = sorted(
    list(paths.list_images(search_dir_anger))
)
image_paths_happy = sorted(
    list(paths.list_images(search_dir_happy))
)
image_paths_normal = sorted(
    list(paths.list_images(search_dir_normal))
)
image_paths_sad = sorted(
    list(paths.list_images(search_dir_sad))
)
image_paths_worry = sorted(
    list(paths.list_images(search_dir_worry))
)

print(">>> total images =", len(image_paths))

print(">>> anger images =", len(image_paths_anger))
print(">>> happy images =", len(image_paths_happy))
print(">>> normal images =", len(image_paths_normal))
print(">>> sad images =", len(image_paths_sad))
print(">>> worry images =", len(image_paths_worry))

In [ ]:
import os
import cv2
from tqdm import tqdm

image_dim = (180, 180, 3)

images = []
labels = []
for image_path in tqdm(image_paths):
    image = cv2.imread(image_path)

    image = cv2.resize(
        image, (image_dim[1], image_dim[0])
    )
    images.append(image)

    label = image_path.split(os.path.sep)[-2]
    labels.append([label])

print(">>> images count =", len(images))

In [ ]:
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

images = np.array(images, dtype='float32') / 255.0
labels = np.array(labels)

mlb = MultiLabelBinarizer()
enc_labels = mlb.fit_transform(labels)

print(f'label shape = {labels}')
print(f'mlb 값 : {mlb}')
print(f'enc_labels : {enc_labels}')
print(">>> classes name =", mlb.classes_)

In [ ]:
from sklearn.model_selection import train_test_split

seed = 47

(x_train, x_test, y_train, y_test) = train_test_split(
    images, enc_labels, test_size=0.2, random_state=seed
)
print(">> train test shape = {} {}".format(
    x_train.shape, y_train.shape)
)

In [ ]:
# import하여 모델 구현

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.applications import ResNet50

# ResNet50 모델 불러오기 (ImageNet 사전 훈련된 가중치 사용, 최상위 레이어 포함하지 않음)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(180, 180, 3))

# 모델의 출력 레이어 제거 및 새로운 출력 레이어 추가
x = base_model.output
x = GlobalAveragePooling2D(keepdims=True)(x)  # keepdims=True 설정
x = Flatten()(x)  # 1차원으로 평탄화
output = Dense(len(mlb.classes_), activation='softmax')(x)  # 분류할 클래스 수에 맞게 설정

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=output)

# 모델 구조 확인
model.summary()


In [ ]:
# 데이터 증강 설정
from tensorflow.keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1, height_shift_range=0.1,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest'
)

In [ ]:
# 옵티마이저 및 손실 함수 설정
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
learning_rate = 1e-3
optimizer = Adam(
    learning_rate=learning_rate,
    beta_1=0.9, beta_2=0.999, epsilon=1e-07
)
loss = CategoricalCrossentropy(from_logits=False)

In [ ]:
# 모델 컴파일
model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=['accuracy']
)

In [ ]:
# 모델 요약 출력
model.summary()

In [ ]:
history = model.fit(
    aug.flow(x_train, y_train, batch_size=128),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // 128,
    epochs=200, verbose=1
)

In [ ]:
test_image_paths = sorted(
    list(
        paths.list_images("") #유효성 검사를 위한 이미지를 불러올 파일 경로를 입력하세요
    )
)
print(">>> test image path =", test_image_paths)

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import numpy as np

print(">>> class index =", mlb.classes_)

for image_path in test_image_paths:
    test_image = cv2.imread(image_path)

    if test_image is None:
        print(f"Error loading image {image_path}")
        continue

    # 이미지 리사이징
    test_image = cv2.resize(test_image, (180, 180))  # 모델이 예상하는 크기로 조정

    cv2_imshow(test_image)

    # 이미지 전처리
    test_image = test_image.astype("float32") / 255.0
    test_image = np.expand_dims(test_image, axis=0)

    try:
        # 모델 예측
        proba = model.predict(test_image)[0]
        print(np.round(proba, 3))
        idx = np.argmax(proba)
        print(">>> predict class =", mlb.classes_[idx])
        print(f">>> actual class = {image_path.split(os.path.sep)[-1]}")
    except Exception as e:
        print(f"Error predicting image {image_path}: {e}")

In [ ]:
# 모델 전체 저장
model.save('')# 모델을 저장할 경로를 입력하세요


In [ ]:
# 가중치만 저장
model.save_weights('') # 가중치를 저장할 경로를 입력하세요